# **Domain Generator — Business Dataset Creation** 

## **Domain Dataset Generation (Groq API)**

To create the dataset, I used an Grop API that is free for now. I chose the LLaMA 3 70B model.

In [ ]:
from create_datasets.domain_dataset import DomainDataset

generator = DomainDataset(from_scratch=False)
generator.generate(n=5)